# 🔄 Post-Mix Integration Notebook
## Mixed Stems → Post-Mix Mastering Pipeline

This notebook integrates your mixed stems with the existing post-mix mastering pipeline.

**Workflow:**
1. Load mixed stems from mixing session
2. Format for post-mix pipeline compatibility
3. Run through all mastering variants (RadioReady, 3D_Immersive, etc.)
4. Generate final masters and previews

In [ ]:
# Core imports
import os
import json
import numpy as np
import soundfile as sf
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🔄 Post-Mix Integration System Ready!")
print("📁 Loads mixed stems and runs mastering pipeline")
print("🎯 All 30+ variants available")

## 📁 Step 1: Load Mixed Stems

Load stems from your mixing session or specify paths directly.

In [ ]:
# Option 1: Load from mixing session JSON
mixing_session_dir = "/Users/itay/Documents/post_mix_data/mixing_sessions/session_XXXXXXXX_XXXXXX"  # Update with your session
stems_json_path = os.path.join(mixing_session_dir, "stems_for_postmix.json")

if os.path.exists(stems_json_path):
    with open(stems_json_path, 'r') as f:
        stem_paths = json.load(f)
    print(f"✅ Loaded stems from: {stems_json_path}")
else:
    # Option 2: Manual stem paths
    stem_paths = {
        "drums_path": "/path/to/drums.wav",
        "bass_path": "/path/to/bass.wav", 
        "vocals_path": "/path/to/vocals.wav",
        "music_path": "/path/to/music.wav",
    }
    print("⚠️ Using manual stem paths - update the paths above")

# Verify all stems exist
print("\n📊 Stem Status:")
all_stems_exist = True
for stem_type, path in stem_paths.items():
    if path and os.path.exists(path):
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"  ✅ {stem_type}: {os.path.basename(path)} ({size_mb:.1f} MB)")
    else:
        print(f"  ❌ {stem_type}: File not found - {path}")
        all_stems_exist = False

if all_stems_exist:
    print("\n✅ All stems ready for post-mix processing")
else:
    print("\n⚠️ Some stems missing - update paths and re-run")

## 🎛️ Step 2: Initialize Post-Mix Pipeline

In [ ]:
# Import the post-mix mastering system
try:
    from mastering_orchestrator import MasteringOrchestrator
    from config import CONFIG
    print("✅ Post-mix system imported successfully")
    
    # Set stem mastering mode
    CONFIG.pipeline.default_mode = CONFIG.pipeline.STEM_MASTERING
    print("✅ Stem mastering mode activated")
    
    # Create workspace for processing
    from io_audio import make_workspace
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    workspace_dir = f"/Users/itay/Documents/post_mix_data/mastering_sessions/postmix_{timestamp}"
    workspace_paths = make_workspace(workspace_dir)
    
    print(f"📁 Workspace created: {workspace_dir}")
    
except ImportError as e:
    print(f"❌ Could not import post-mix system: {e}")
    print("⚠️ Make sure you're in the correct directory with all modules")

## 🎯 Step 3: Configure Mastering Settings

Choose which variants and mastering styles you want to process.

In [ ]:
# Configure which variants to process
variants_to_process = [
    # Core variants
    "Stem_Original",
    "Stem_RadioReady", 
    "Stem_Bassier",
    "Stem_Punchier",
    "Stem_Wider",
    
    # Hybrid variants (advanced + depth)
    "Stem_RadioReady_Depth",
    "Stem_Aggressive_Depth",
    "Stem_PunchyMix_Depth",
    
    # 3D and immersive
    "Stem_3D_Immersive",
    "Stem_3D_Wide",
    
    # Add more as needed...
]

# Mastering styles to apply
mastering_styles = [
    "Neutral",    # Clean, transparent
    "Warm",       # Warm, musical
    "Bright",     # Clear, present
    "Loud",       # Competition-ready
]

print(f"🎯 Processing {len(variants_to_process)} stem variants")
print(f"🎚️ Applying {len(mastering_styles)} mastering styles")
print(f"📊 Total combinations: {len(variants_to_process) * len(mastering_styles)}")

## 🚀 Step 4: Run Post-Mix Processing

In [ ]:
if all_stems_exist:
    try:
        # Initialize orchestrator with workspace
        from io_audio import Manifest
        manifest = Manifest()
        
        orchestrator = MasteringOrchestrator(
            workspace_paths=workspace_paths,
            manifest=manifest
        )
        
        print("✅ MasteringOrchestrator initialized")
        
        # Register the input stems
        from io_audio import register_input
        
        input_stems = {}
        for stem_type, path in stem_paths.items():
            if path and os.path.exists(path):
                stem_name = stem_type.replace('_path', '')
                input_stems[stem_name] = register_input(path, manifest, workspace_paths)
                print(f"  📁 Registered {stem_name}: {os.path.basename(path)}")
        
        print(f"\n🎛️ Starting mastering process...")
        print(f"  • {len(input_stems)} stems")
        print(f"  • {len(variants_to_process)} variants")
        print(f"  • {len(mastering_styles)} mastering styles")
        print("\n⏳ This may take several minutes...\n")
        
        # Process each variant and mastering style combination
        results = {}
        total_combinations = len(variants_to_process) * len(mastering_styles)
        current = 0
        
        for variant in variants_to_process:
            for style in mastering_styles:
                current += 1
                combo_name = f"{variant}_{style}"
                
                print(f"  [{current:2d}/{total_combinations}] Processing {combo_name}...")
                
                try:
                    # This is where you'd call the actual mastering process
                    # The exact API depends on your MasteringOrchestrator implementation
                    result = orchestrator.process_variant(
                        stems=input_stems,
                        variant_name=variant,
                        mastering_style=style,
                        output_name=combo_name
                    )
                    
                    results[combo_name] = result
                    print(f"       ✅ {combo_name} completed")
                    
                except Exception as e:
                    print(f"       ❌ {combo_name} failed: {e}")
                    results[combo_name] = {"error": str(e)}
        
        print(f"\n✅ Post-mix processing complete!")
        print(f"📁 Results saved to: {workspace_dir}")
        
        # Save processing summary
        summary = {
            "timestamp": timestamp,
            "input_stems": stem_paths,
            "variants_processed": variants_to_process,
            "mastering_styles": mastering_styles,
            "total_outputs": len([r for r in results.values() if "error" not in r]),
            "errors": len([r for r in results.values() if "error" in r]),
            "results": results,
            "workspace": workspace_dir
        }
        
        summary_file = os.path.join(workspace_dir, "postmix_summary.json")
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2)
        
        print(f"📄 Summary saved: {summary_file}")
        
    except Exception as e:
        print(f"❌ Post-mix processing failed: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("⚠️ Cannot proceed - missing stem files")

## 📊 Step 5: Review Results

In [ ]:
# Review the processed results
try:
    print("📊 Processing Results:")
    
    successful = [name for name, result in results.items() if "error" not in result]
    failed = [name for name, result in results.items() if "error" in result]
    
    print(f"\n✅ Successful: {len(successful)}")
    for name in successful[:10]:  # Show first 10
        print(f"  • {name}")
    if len(successful) > 10:
        print(f"  ... and {len(successful) - 10} more")
    
    if failed:
        print(f"\n❌ Failed: {len(failed)}")
        for name in failed:
            error = results[name].get("error", "Unknown error")
            print(f"  • {name}: {error}")
    
    print(f"\n📁 All outputs saved to: {workspace_dir}")
    print("🎯 Your masters are ready!")
    
    # List output files
    output_files = list(Path(workspace_dir).glob("**/*.wav"))
    if output_files:
        total_size = sum(f.stat().st_size for f in output_files) / (1024 * 1024)
        print(f"\n📈 Generated {len(output_files)} audio files ({total_size:.1f} MB total)")
    
except NameError:
    print("⚠️ No results to display - processing may not have completed")

## 🎵 Step 6: Generate Previews (Optional)

In [ ]:
# Generate streaming previews for easy comparison
try:
    if 'orchestrator' in locals() and successful:
        print("🎵 Generating preview files...")
        
        # Create preview versions (30-second clips, normalized)
        preview_dir = os.path.join(workspace_dir, "previews")
        os.makedirs(preview_dir, exist_ok=True)
        
        for result_name in successful[:5]:  # Preview first 5 results
            try:
                # This would use your existing preview generation system
                preview_path = orchestrator.create_preview(
                    result_name, 
                    output_dir=preview_dir,
                    duration=30,
                    start_time=60  # Start 1 minute in
                )
                print(f"  ✅ Preview: {os.path.basename(preview_path)}")
            except Exception as e:
                print(f"  ⚠️ Preview failed for {result_name}: {e}")
        
        print(f"\n📁 Previews saved to: {preview_dir}")
    else:
        print("⚠️ No successful results to preview")
        
except Exception as e:
    print(f"❌ Preview generation failed: {e}")

## ✅ Integration Complete!

Your mixed stems have been processed through the complete post-mix mastering pipeline.

**What you now have:**
- Multiple mastered versions of your mix
- Different sonic characteristics (RadioReady, 3D_Immersive, etc.)
- Various mastering styles (Neutral, Warm, Bright, Loud)
- Preview files for easy comparison
- Complete processing logs

**Next steps:**
1. Listen to the different versions
2. Choose your favorites
3. Use for streaming, radio, or distribution
4. Archive the session for future reference